In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

class style :
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# check python bit version
import platform
print(platform.architecture())

!pip install jupyternotify
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

!pip install email-notify-magic
%load_ext email_notify_magic

('64bit', 'WindowsPE')


<IPython.core.display.Javascript object>

In [2]:
%%notify
%%time

Review= pd.read_excel(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Products.xlsx", sheet_name=3, header=0)
Review= Review[['ProductID', 'ReviewText', 'ReviewStar']]
# Review['ReviewText']=Review['ReviewText'].apply(str).apply(lambda x: clean(x))
# Review.to_csv(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Review.csv")
print(Review.shape)

(50887, 3)
Wall time: 33.1 s


<IPython.core.display.Javascript object>

## Cleaning: Code

In [3]:
%%notify
%%time

from nltk.corpus import wordnet
from nltk import word_tokenize

def clean(text):
    
    text_token = word_tokenize(text)
    text_token2 = [x.lower() for x in text_token]
    
    # Mapping Social-media shortcuts
    shortcuts = {'u': 'you','lol': 'laughing out loud', 'k': 'okay', 'm': 'am', 'b4': 'before',
                'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love','gr8': 'great', 'bday': 'birthday',
                 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate','irl': 'in real life', 'yolo': 'you only live once','btw': 'by the way'}
    nsw=pd.Series(text_token)
    words=nsw.map(shortcuts).fillna(nsw)
    
    #Get an integrated list of listed_words (words like yolo must be combined, You Only Live Once :)
    words= [word.split(' ') for word in words]    
    import itertools
    words= list(itertools.chain.from_iterable(words))
    
    # lemmatize the words with appropriate POS Tag
    def pos_tag(word):  
        tag= nltk.pos_tag([word])[0][1][0].upper()
        dic= {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
        return dic.get(tag, wordnet.NOUN)
    words = [WordNetLemmatizer().lemmatize(word, pos=pos_tag(word)) for word in words] 
    words2=[word for word in words if nltk.pos_tag([word])[0][1] not in ['VB','VBG','VBD','VBN','VBP','VBZ','CC','MD','IN']]
    
    '''# perform stemming for non lemmatized words (like 'cleaning')
    from nltk.stem.snowball import SnowballStemmer
    
    englishStemmer=SnowballStemmer("english")
    words=[englishStemmer.stem(word) for  word in words]'''
    
    '''words=[word.encode('utf-8') for word in words]
    words=[hnspl.suggest(word)[0] if hnspl.spell(word)==False else word for word in words]'''
    
    clean_text=" ".join(words2)
    return clean_text

Wall time: 1.23 s


<IPython.core.display.Javascript object>

In [4]:
# !pip install spacy --user
!python -m spacy download en

2021-04-19 18:29:18.423534: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-04-19 18:29:18.424873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from spacy.lang.en import English

import en_core_web_sm
nlp = en_core_web_sm.load()

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishabh.pandey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Spacy&TextBlob: ABSA

In [6]:
# token the sentences
import en_core_web_sm
nlp = en_core_web_sm.load()

from textblob import TextBlob
def ABSA(text):
    aspects= []
    sentences= nltk.sent_tokenize(text)
    for sentence in sentences:
        doc = nlp(sentence)
        descriptive_term = ''
        target = ''
        for token in doc:
            if token.dep_ == 'nsubj' and token.pos_ in ['NOUN','PROPN']:
                target = token.text
            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ != 'ADV':
                        continue
                    prepend += child.text + ' '
                descriptive_term = prepend + token.text
        aspects.append({'target': target,'adjective': descriptive_term})

        for aspect in aspects:
            aspect['text']= aspect['target']+' '+aspect['adjective']
            aspect['polarity'] = TextBlob(aspect['adjective']).sentiment.polarity
            # aspect['subjectivity'] = TextBlob(aspect['adjective']).sentiment.subjectivity
    return aspects

## Test

In [11]:
text="Camera of Nokia is very great!"
ABSA(text)

[{'target': 'Camera',
  'adjective': 'very great',
  'text': 'Camera very great',
  'polarity': 1.0}]

In [16]:
text="Samsung has an extremely bad battery but good camera"
ABSA(text)

[{'target': 'Samsung',
  'adjective': 'good',
  'text': 'Samsung good',
  'polarity': 0.7}]